In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/inria-bci-challenge/TrainLabels.csv
/kaggle/input/inria-bci-challenge/train.zip
/kaggle/input/inria-bci-challenge/SampleSubmission.csv
/kaggle/input/inria-bci-challenge/ChannelsLocation.csv
/kaggle/input/inria-bci-challenge/test.zip
/kaggle/input/truelabels/true_labels.csv


In [ ]:
! pip install mne
! pip install pyriemann

In [ ]:
import zipfile

# Will unzip the files so that you can see them..
with zipfile.ZipFile("../input/inria-bci-challenge/train.zip","r") as z:
    z.extractall("./train")

In [ ]:
with zipfile.ZipFile("../input/inria-bci-challenge/test.zip","r") as z:
    z.extractall("./test")

In [ ]:
import numpy as np                                      # for dealing with data
from scipy.signal import butter, sosfiltfilt, sosfreqz  # for filtering
import matplotlib.pyplot as plt                         # for plotting
from scipy import interp
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
import os
from os import listdir
from os.path import isfile, join, isdir
import pickle

In [ ]:
# Create filtering variables
fs = 200.0     # 200 Hz sampling rate
lowcut = 1.0   # 0.1 Hz is the lowest frequency we will pass
highcut = 40.0 # 40 Hz is the highest frequency we will pass.

In [ ]:
def butter_bandpass_filter(raw_data, fs, lowcut = 1.0, highcut =40.0, order = 5):
    '''
    The filter I want to apply to my raw eeg data.
    :raw_data (nparray): data you want to process
    :fs (float): sampling rate
    :lowcut (float, optional): lowest frequency we will pass
    :highcut (float, optional): highest frequency we will pass
    :order (int, optional): order of filter
    '''
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog = False, btype = 'band', output = 'sos')
    filted_data = sosfiltfilt(sos, raw_data)
    return filted_data

In [ ]:
epoch_s = 0      # epoch starting time relative to stmulus in miliseconds
epoch_e = 700    # epoch ending time relative to stmulus in miliseconds
bl_s = 0         # baseline starting time relative to stmulus in miliseconds
bl_e = 100       # baseline ending time relative to stmulus in miliseconds


# number of mark per epoch
epoch_len = int((abs(epoch_s) + abs(epoch_e)) * (fs / 1000))

In [ ]:
train_subj_num = 16
test_subj_num = 10
stimulus_per_subj = 340
trial_per_subj = 5

channels = ['Fp1', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8', 'F7', 'F5', 'F3', 'F1',
    'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz',
    'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2',
    'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4',
    'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8',
    'PO7', 'POz', 'P08', 'O1', 'O2']

In [ ]:
import pandas as pd
train_labels = pd.read_csv("../input/inria-bci-challenge/TrainLabels.csv")
sample_train_data = pd.read_csv("./train/Data_S02_Sess01.csv")

In [ ]:
train_labels

,IdFeedBack,Prediction
0,S02_Sess01_FB001,1
1,S02_Sess01_FB002,1
2,S02_Sess01_FB003,0
3,S02_Sess01_FB004,0
4,S02_Sess01_FB005,1
...,...,...
5435,S26_Sess05_FB096,1
5436,S26_Sess05_FB097,0
5437,S26_Sess05_FB098,0
5438,S26_Sess05_FB099,0


In [ ]:

df2 = sample_train_data[sample_train_data['FeedBackEvent'] == 1]

In [ ]:
df2


,Time,Fp1,Fp2,AF7,AF3,AF4,AF8,F7,F5,F3,...,P4,P6,P8,PO7,POz,P08,O1,O2,EOG,FeedBackEvent
9599,47.995,830.677222,979.638619,847.257758,766.929505,555.929311,853.074414,777.926970,910.416082,798.451960,...,568.057948,817.546838,741.133120,742.770663,313.393700,932.304475,750.347476,969.756009,-1591.606547,1
11172,55.860,741.397327,872.008242,766.300141,677.442596,457.186287,782.537999,700.256845,824.570032,711.395305,...,493.392074,757.499946,663.731070,643.358617,280.655005,835.169209,682.361834,985.912887,-1632.253751,1
12755,63.775,655.710247,800.222757,690.103381,608.341656,428.382266,705.809794,657.269119,744.304954,623.569055,...,437.289179,695.405881,609.067129,588.200731,240.277625,735.432159,637.662106,939.623087,-1588.697668,1
14332,71.660,505.521992,656.193543,535.766742,460.644899,295.091007,548.810110,491.984303,573.692467,457.116312,...,307.653602,551.175548,481.569049,443.286527,93.176044,597.832214,487.044101,765.703260,-1471.171030,1
15905,79.525,392.508071,504.661776,395.453906,339.234001,157.797753,389.749396,359.081097,444.971754,326.184913,...,177.583071,372.191496,339.157296,304.684089,-32.193317,449.920405,315.785541,620.761488,-1695.429472,1
18389,91.945,279.300250,386.458820,272.636807,227.050504,88.332005,316.084782,275.894772,334.889786,257.080028,...,95.283786,342.494347,278.072964,227.922950,-59.427562,309.652283,263.941596,472.002025,-882.964562,1
19966,99.830,173.119037,275.487436,168.220275,133.354826,-3.935489,211.151521,173.075003,236.515696,159.796689,...,1.718842,231.785307,179.812840,166.036672,-177.955435,200.031687,164.201359,262.719445,-779.605068,1
21542,107.710,81.258856,186.608261,90.835285,50.701160,-90.263715,115.074201,106.127514,151.640304,61.555243,...,-77.226319,131.510198,87.639915,60.515073,-211.423577,136.675139,80.694990,233.246619,-650.485148,1
23119,115.595,76.591264,172.131912,71.468919,48.363957,-160.143684,94.280169,88.028956,152.045731,52.732909,...,-67.526519,134.463873,109.224401,43.092283,-240.339442,138.519395,73.780588,124.533266,-446.426849,1
24699,123.495,32.246924,123.315569,34.293847,2.574848,-179.837638,73.917818,63.458661,101.183775,15.963456,...,-110.773964,107.545349,73.886629,26.394442,-241.228906,76.930600,36.133601,121.697212,-374.486961,1


In [ ]:
train_list_arr = np.array(sorted(listdir('./train')))
train_list_np = np.reshape(train_list_arr, 
                           (train_subj_num, trial_per_subj))
test_list_arr = np.array(sorted(listdir('./test')))
test_list_np = np.reshape(
    test_list_arr[0:50], (test_subj_num, trial_per_subj))
print(train_list_np.shape       ,test_list_np.shape)

train_data_list = np.empty(
    (0, stimulus_per_subj, len(channels), epoch_len), float)
test_data_list = np.empty(
    (0, stimulus_per_subj, len(channels), epoch_len), float)
print(train_data_list.shape,test_data_list.shape)

(16, 5) (10, 5)
(0, 340, 56, 140) (0, 340, 56, 140)


In [ ]:
train_list_np

array([['Data_S02_Sess01.csv', 'Data_S02_Sess02.csv',
        'Data_S02_Sess03.csv', 'Data_S02_Sess04.csv',
        'Data_S02_Sess05.csv'],
       ['Data_S06_Sess01.csv', 'Data_S06_Sess02.csv',
        'Data_S06_Sess03.csv', 'Data_S06_Sess04.csv',
        'Data_S06_Sess05.csv'],
       ['Data_S07_Sess01.csv', 'Data_S07_Sess02.csv',
        'Data_S07_Sess03.csv', 'Data_S07_Sess04.csv',
        'Data_S07_Sess05.csv'],
       ['Data_S11_Sess01.csv', 'Data_S11_Sess02.csv',
        'Data_S11_Sess03.csv', 'Data_S11_Sess04.csv',
        'Data_S11_Sess05.csv'],
       ['Data_S12_Sess01.csv', 'Data_S12_Sess02.csv',
        'Data_S12_Sess03.csv', 'Data_S12_Sess04.csv',
        'Data_S12_Sess05.csv'],
       ['Data_S13_Sess01.csv', 'Data_S13_Sess02.csv',
        'Data_S13_Sess03.csv', 'Data_S13_Sess04.csv',
        'Data_S13_Sess05.csv'],
       ['Data_S14_Sess01.csv', 'Data_S14_Sess02.csv',
        'Data_S14_Sess03.csv', 'Data_S14_Sess04.csv',
        'Data_S14_Sess05.csv'],
       ['Data_S16_Se

In [ ]:
test_list_np

array([['Data_S01_Sess01.csv', 'Data_S01_Sess02.csv',
        'Data_S01_Sess03.csv', 'Data_S01_Sess04.csv',
        'Data_S01_Sess05.csv'],
       ['Data_S03_Sess01.csv', 'Data_S03_Sess02.csv',
        'Data_S03_Sess03.csv', 'Data_S03_Sess04.csv',
        'Data_S03_Sess05.csv'],
       ['Data_S04_Sess01.csv', 'Data_S04_Sess02.csv',
        'Data_S04_Sess03.csv', 'Data_S04_Sess04.csv',
        'Data_S04_Sess05.csv'],
       ['Data_S05_Sess01.csv', 'Data_S05_Sess02.csv',
        'Data_S05_Sess03.csv', 'Data_S05_Sess04.csv',
        'Data_S05_Sess05.csv'],
       ['Data_S08_Sess01.csv', 'Data_S08_Sess02.csv',
        'Data_S08_Sess03.csv', 'Data_S08_Sess04.csv',
        'Data_S08_Sess05.csv'],
       ['Data_S09_Sess01.csv', 'Data_S09_Sess02.csv',
        'Data_S09_Sess03.csv', 'Data_S09_Sess04.csv',
        'Data_S09_Sess05.csv'],
       ['Data_S10_Sess01.csv', 'Data_S10_Sess02.csv',
        'Data_S10_Sess03.csv', 'Data_S10_Sess04.csv',
        'Data_S10_Sess05.csv'],
       ['Data_S15_Se

In [ ]:
def generate_epoch(file_path, channels, fs, eeg_filter, stimulus_times=None, baseline=True,  epoch_s=0, epoch_e=700, bl_s=0, bl_e=100):
    """
    :description: Generating epoch given csv file. Make sure the csv file layout meets the requirement.
        It should contain 'Time' column that represents timepoints, and the time should start from 0.
        If your csv file does not have FeedBackEvent indicating the stimulus, you must pass stumulus_times.
        Here we used a butter bandpass filter, but you can change to your favorite one.

    :file_path (String): path to your csv file
    :channels ([String]): array of channels to epoch
    :fs (float): sampling rate
    :eeg_filter (function): the filter you want to apply to raw eeg data
    :stimulus_times ([float], optional): The time points that stimulus occur
    :baseline (boolean, optional): whether you want to apply baseline correction after epoching
    :epoch_s (int, optional): epoch starting time relative to stmulus in miliseconds
    :epoch_e (int, optional): epoch ending time relative to stmulus in miliseconds
    :bl_s (int, optional): baseline starting time relative to stmulus in miliseconds
    :bl_e (int, optional): baseline ending time relative to stmulus in miliseconds

    :rtype (3d-nparray): epoched data with dimension (stimulus_per_trial, number_of_channels, number_of_time_points)
    """
    # read data and data selection
    train_data = pd.read_csv(file_path)

    train_data.loc[:, 'Time'] = train_data.loc[:, 'Time']*1000
    raw_eeg = train_data[channels].values.T

    time_df = train_data['Time'].values
    train_data['index'] = train_data.index.values
    if stimulus_times is None:
        mark_indices = np.asarray(
            train_data[train_data['FeedBackEvent'] == 1].index).flatten()
    else:
        mark_indices = np.round(np.asarray(
            stimulus_times).flatten() * fs).astype(int)

    # Define the bounds of our epoch as well as our baseline
    # index in epoch_df where our baseline begins
    b_s = int((abs(epoch_s) + bl_s) * (fs / 1000))
    # index in epoch_df where our baseline ends
    b_e = int((abs(epoch_s) + bl_e) * (fs / 1000))
    # Let's calculate the length our epoch with our given sampling rate
    epoch_len = int((abs(epoch_s) + abs(epoch_e)) * (fs / 1000))

    # Let's define some helpful variables to make our extraction easier
    # effectively the number of indices before marker we want
    e_s = int((epoch_s * (fs / 1000)))
    # effectively the number of indices after marker we want
    e_e = int((epoch_e * (fs / 1000)))

    # Epoch the data
    final_epoch = np.empty((mark_indices.shape[0], epoch_len, 0), float)
    for channel in channels:
        epoch = np.zeros(shape=(int(mark_indices.shape[0]), epoch_len))
        raw_eeg = train_data[channel].values

        ################# You may want to apply your own filter ################
        clean_eeg = eeg_filter(raw_eeg, fs, lowcut, highcut, 5)
        ########################################################################

        for i, mark_idx in enumerate(mark_indices):
            # grab the appropriate samples around the stimulus onset
            epoch[i, :] = clean_eeg[mark_idx + e_s: mark_idx + e_e]

        # Baseline correction
        if baseline:
            for i in range(0, int(epoch.shape[0])):
                epoch[i, :] = epoch[i, :] - np.mean(epoch[i, b_s:b_e])

        # stack epoch of each channel
        final_epoch = np.dstack((final_epoch, epoch))
    final_epoch = np.swapaxes(final_epoch, 1, 2)
    return final_epoch

In [ ]:
if not isfile("./train_data.npy"):
    for training_participant_id in range(train_subj_num):
        subject_dir_list = train_list_np[training_participant_id]
        subject_epoch = np.empty((0, len(channels), epoch_len), float)
        index=1
        for trial_id in range(trial_per_subj):
            subject_dir = subject_dir_list[trial_id]
            data = generate_epoch('./train/'+subject_dir, channels, fs,
                butter_bandpass_filter, epoch_s = epoch_s, epoch_e = epoch_e, bl_s = bl_s, bl_e = bl_e)
            subject_epoch = np.vstack((subject_epoch, data))
        print(subject_epoch.shape)
        subject_epoch = np.reshape(
            subject_epoch, (index, stimulus_per_subj, len(channels), epoch_len))
        train_data_list = np.vstack((train_data_list, subject_epoch))
        index=index+1

    print('Epoched training data shape: ' + str(train_data_list.shape))

    for testing_participant_id in range(test_subj_num):
        subject_dir_list = test_list_np[testing_participant_id]
        subject_epoch = np.empty((0, len(channels), epoch_len), float)
        for trial_id in range(trial_per_subj):
            subject_dir = subject_dir_list[trial_id]
            data = generate_epoch('./test/'+subject_dir, channels, fs,
                butter_bandpass_filter, epoch_s = epoch_s, epoch_e = epoch_e, bl_s = bl_s, bl_e = bl_e)
            subject_epoch = np.vstack((subject_epoch, data))
        subject_epoch = np.reshape(
            subject_epoch, (1, stimulus_per_subj, len(channels), epoch_len))
        test_data_list = np.vstack((test_data_list, subject_epoch))

    print('Epoched testing data shape: ' + str(test_data_list.shape))

    np.save('./train_data.npy', train_data_list)
    np.save('./test_data.npy', test_data_list)

(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
(340, 56, 140)
Epoched training data shape: (16, 340, 56, 140)
Epoched testing data shape: (10, 340, 56, 140)


In [ ]:
train_data_list = np.load('./train_data.npy')
test_data_list = np.load('./test_data.npy')
print('Epoched training data shape: ' + str(train_data_list.shape))
print('Epoched testing data shape: ' + str(test_data_list.shape))

Epoched training data shape: (16, 340, 56, 140)
Epoched testing data shape: (10, 340, 56, 140)


In [ ]:
pip install -U numpy scipy scikit-learn

     |████████████████████████████████| 15.3 MB 133 kB/s eta 0:00:01
     |████████████████████████████████| 27.4 MB 35.7 MB/s eta 0:00:01
     |████████████████████████████████| 22.3 MB 31.5 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.4
    Uninstalling scipy-1.5.4:
      Successfully uninstalled scipy-1.5.4
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.20.3 which is incompatible.
tensorflow 2.4.1 requires numpy~=1.19

In [ ]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace

In [ ]:
Y_train = pd.read_csv('../input/inria-bci-challenge/TrainLabels.csv')['Prediction'].values

In [ ]:
train_data_list.shape

(16, 340, 56, 140)

In [ ]:
print(stimulus_per_subj,len(channels),epoch_len)

340 56 140


In [ ]:
if not isfile('./X_train.npy'):
    XC = XdawnCovariances(nfilter=5) # our transformer
    TS = TangentSpace(metric='riemann')
    # collapse first two dimension
    train_data = np.reshape(train_data_list, 
    (16 * stimulus_per_subj, len(channels), epoch_len))
    test_data = np.reshape(test_data_list, 
    (test_subj_num * stimulus_per_subj, len(channels), epoch_len))

    # transform our data
    X_train = XC.fit_transform(train_data, Y_train)
    X_train = TS.fit_transform(X_train)
    X_test = XC.transform(test_data)
    X_test = TS.transform(X_test)

    # save to local
    np.save('./X_train', X_train)
    np.save('./X_test', X_test)
    np.save('./Y_train', Y_train)

In [ ]:
# Load our data
X_train = np.load('./X_train.npy')
X_test = np.load('./X_test.npy')
Y_train = np.load('./Y_train.npy')

# Only used for scoring
Y_test = np.reshape(pd.read_csv('../input/truelabels/true_labels.csv', header=None).values, 3400)

In [ ]:
print('Transformed training data shape: ' + str(X_train.shape))
print('Training label shape: ' + str(Y_train.shape))
print('Transformed testing data shape: ' + str(X_test.shape))
print('Testing label shape: ' + str(Y_test.shape))

Transformed training data shape: (5440, 210)
Training label shape: (5440,)
Transformed testing data shape: (3400, 210)
Testing label shape: (3400,)


XGBoost Classifier

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, Y_train)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:38:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [ ]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [ ]:
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 73.71%
